In [1]:
import matplotlib.pyplot as plt
from matrix_control import transposeMatrix, printMatrix, reshapeMatrix, sumMatrix
from layer import Neuron
from model import LinearModel

In [2]:
train_image = []
sample_num = 14
for i in range(sample_num):
    train_image.append(plt.imread(fname='image/train/train{}.jpeg'.format(i+1)))

In [3]:
label = ['t', 'u', 'v', 'w', 'x', 'y', 'z', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
train_image[3]

array([[ 3,  0,  6,  0,  2,  1,  0,  6,  0,  3,  1,  4,  0,  3,  0,  1],
       [ 0,  0,  2,  0,  0,  5,  0,  0,  0,  1,  0,  0,  6,  2,  2,  0],
       [ 2,  3,  0,  0,  6,  0,  4,  4, 17,  1,  3,  0,  0,  1,  0,  2],
       [11, 25,  0,  0,  5,  0,  1, 29, 65,  2,  1,  0, 11, 31,  3,  0],
       [10, 48,  3,  5,  0,  3,  1, 49, 66,  3,  0,  2, 35, 34,  1,  0],
       [ 1, 42, 25,  0,  1,  1, 19, 42, 36, 16,  0,  2, 47,  5,  0,  5],
       [ 0, 16, 44,  1,  6,  0, 41, 19, 31, 31,  0, 12, 47,  1,  2,  2],
       [ 2,  0, 44, 20,  0,  5, 50,  3, 24, 29,  3, 29, 29,  2,  3,  0],
       [ 0,  0, 11, 47,  4, 17, 40,  0,  7, 47,  3, 45, 11,  2,  0,  2],
       [ 0,  2,  2, 35, 24, 35, 22,  1,  0, 39, 17, 47,  4,  0,  2,  0],
       [ 2,  0,  1, 13, 59, 49, 12,  0,  1, 32, 54, 43,  0,  0,  5,  0],
       [ 2,  0,  4,  0, 55, 36,  4,  0,  1, 20, 74, 24,  0,  1,  4,  0],
       [ 0,  5,  1,  0, 11,  8,  0,  2,  0,  6, 44,  5,  0,  2,  0,  1],
       [ 0,  2,  2,  1,  0,  0,  2,  0,  0,  3,  0,

In [5]:
input_size = 256
hidden_size_1 = 96
hidden_size_2 = 48
output_size = 7
learning_rate = 0.0001

In [6]:
model = LinearModel(input_size=input_size, hidden_size1=hidden_size_1, hidden_size2=hidden_size_2, output_size=output_size, sample_num=sample_num, learning_rate=learning_rate)
model.forward(train_image, label)

모델 생성 시작. 입력차원 256개, 1히든차원 96개, 2히든차원 48개, 출력층 7개로, 샘플 개수는 14개 입니다.
output size : 96 X 14
output size : 48 X 14
output size : 7 X 14
[[1135.6678194347328], [1230.9906668514645], [1166.8105732506676], [1255.9900011775], [1145.2823739147063], [1206.5049049152783], [1187.9463248780805]]
[[0.08743282020910062, 0.09269709691084219, 0.05968845584624745, 0.0573997815533556, 0.08124470570197162, 0.05294984010747239, 0.08339819065719219, 0.019478021059108775, 0.015603829349503674, 0.005739188600945167, 0.08048121225134237, 0.02036502223888752, 0.04419441986770352, 0.07783695048220791, 0.09200163487796875, 0.0066417594526033685, 0.027911724841326892, 0.07187605022500074, 0.05617105997777179, 0.06406274564540058, 0.00010417891717042772, 0.016143735103411383, 0.02573323679494164, 0.08446140590787221, 0.03882676410579721, 0.07286228167148495, 0.06849573000834923, 0.03209030982310092, 0.06185065578987792, 0.0115029132895625, 0.08150991685966802, 0.06357420757726169, 0.07987669037282935, 0.0496596533

ValueError: Number of columns in the first matrix must be equal to the number of rows in the second matrix.